In [1]:
import pandas as pd

from typing import List, Tuple
from dotenv import load_dotenv
import os
from typing import Callable

load_dotenv()
raw_data_folder = os.getenv("rawDataDirectory")
data_folder = os.getenv("dataFolder")
privateLatStr = os.getenv("privateLat")
privateLonStr = os.getenv("privateLon")
privateRadStr = os.getenv("privateRad")

assert (
    privateLatStr is not None
    and privateLonStr is not None
    and privateRadStr is not None
)

privateLat = float(privateLatStr)
privateLon = float(privateLonStr)
privateRad = float(privateRadStr)

from util.clean_file import filter_by_distance

filterFunction: Callable[[float, float], bool] = lambda lat, lon: filter_by_distance(
    (lat, lon), (privateLat, privateLon, privateRad)
)

In [2]:
def read_good_trace():
    locations = pd.read_csv(
        f"{raw_data_folder}/data_18_02",
        delimiter=";",
        header=0,
        dtype={
            "finePermission": bool,
            "foreGround": bool,
            "priority": int,
            "user": str,
            "time": int,
            "realLocation": str,
            "obfuscatedLocation": str,
        },
    )

    locations["time"] = pd.to_datetime(locations["time"], unit="ms")
    goodTrace = locations[locations["user"] == "bd9df063-0d45-4e20-b604-7eb16d0954f5"]
    real_location_tuples: List[Tuple[float, float, int]] = (
        goodTrace["realLocation"].apply(eval).tolist()
    )
    return real_location_tuples

In [3]:
trace = read_good_trace()
print(len(trace))

trace = [loc for loc in trace if filterFunction(loc[0], loc[1])]
print(len(trace))

with open(f"{data_folder}/good_trace", "w") as file:

    for item in trace:

        file.write(f"{item[0]}, {item[1]}, {item[2]}\n")

10889
10740
